## Load Data

In [3]:
import pandas as pd
import numpy as np

original_data = pd.read_excel('2017임산물생산조사.xlsx')
original_data.head(3)

,시도/청,시군구/관리소,임가번호,구분(재배/채취),구분(재배/채취).1,구분(재배/채취).2,Unnamed: 6,구분(재배/채취).3,품목,품목.1,품목.2,품목.3,최종값,Unnamed: 13,Unnamed: 14,비고,점검사항,등록자,최초등록일,최종수정일
0,NaN,NaN,NaN,재배채취,재배기준,재배면적,환산면적,재배단위,대분류,중분류,소분류,품목단위,생산량,단가,생산액,NaN,NaN,NaN,NaN,NaN
1,서울특별시,종로구,341235.0,채취,NaN,0,0,㎡,수실,재배수실,은행,kg,36920,3000,110760000,3영급 (20년생 이상)의 나무,NaN,A1001P101,2018-04-24,2018-04-24
2,서울특별시,중구,341243.0,채취,NaN,0,0,㎡,수실,재배수실,은행,kg,22720,3000,68160000,3영급 (20년생 이상)의 나무,NaN,A1001P101,2018-04-24,2018-04-24


In [4]:
data = original_data.iloc[1:, [1, 3, 5, 7, 10, 12]]
data.rename(columns={'구분(재배/채취)' : '재배채취',
                     '구분(재배/채취).1' : '재배기준',
                     '구분(재배/채취).2' : '재배면적',
                     '구분(재배/채취).3' : '재배단위',
                     'Unnamed: 6' : '환산면적',
                     '품목' : '품목 대분류',
                     '품목.1' : '품목 중분류',
                     '품목.2' : '품목 소분류',
                     '최종값' : '생산량'},
            inplace = True)
data.head()

,시군구/관리소,재배채취,재배면적,재배단위,품목 소분류,생산량
1,종로구,채취,0,㎡,은행,36920
2,중구,채취,0,㎡,은행,22720
3,용산구,채취,0,㎡,은행,41440
4,성동구,채취,0,㎡,은행,9960
5,광진구,채취,0,㎡,은행,12800


In [5]:
data = data[data['품목 소분류'].isin(['밤', '호두', '대추', '떫은감', '건표고', '생표고', '더덕', '오미자'])]
data

,시군구/관리소,재배채취,재배면적,재배단위,품목 소분류,생산량
172,달성군,재배,10,평,더덕,5
203,달성군,재배,150,평,오미자,20
213,달성군,재배,400,평,오미자,15
247,달성군,재배,400,평,오미자,5
285,달성군,재배,40,평,오미자,20
...,...,...,...,...,...,...
106361,서귀포시,재배,246404,봉,생표고,163775
106362,제주시,재배,99500,㎡,떫은감,1464
106363,제주시,재배,297033,봉,건표고,21140
106364,제주시,재배,297033,봉,생표고,47394


In [6]:
data.dropna(axis=0, inplace=True)

data['재배면적'] = np.log1p(data['재배면적'].astype(float))
data['생산량'] = np.log1p(data['생산량'].astype(float))
data.head()

,시군구/관리소,재배채취,재배면적,재배단위,품목 소분류,생산량
172,달성군,재배,2.397895,평,더덕,1.791759
203,달성군,재배,5.017280,평,오미자,3.044522
213,달성군,재배,5.993961,평,오미자,2.772589
247,달성군,재배,5.993961,평,오미자,1.791759
285,달성군,재배,3.713572,평,오미자,3.044522


In [7]:
from sklearn.preprocessing import MinMaxScaler
# column 별로 scaling 되는 것이 아님 주의

X_minmax_scaler = MinMaxScaler()
X_minmax_scaler.fit(data.loc[:, ['재배면적']])
X_scaled = X_minmax_scaler.transform(data.loc[:, ['재배면적']])
X_scaled

array([[0.13299905],
       [0.27828299],
       [0.33245455],
       ...,
       [0.69894675],
       [0.69894675],
       [0.33408731]])

In [8]:
y_minmax_scaler = MinMaxScaler()
y_minmax_scaler.fit(data.loc[:, ['생산량']])
y_scaled = y_minmax_scaler.transform(data.loc[:, ['생산량']])
y_scaled

array([[0.10166934],
       [0.17275453],
       [0.15732427],
       ...,
       [0.56509918],
       [0.61090773],
       [0.6424904 ]])

In [9]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoder.fit(data.loc[:, ['시군구/관리소', '재배채취', '재배단위', '품목 소분류']])
onehot = onehot_encoder.fit_transform(data.loc[:, ['시군구/관리소', '재배채취', '재배단위', '품목 소분류']])
print(onehot.toarray().shape)

(8571, 181)


In [10]:
numeric_data_df = pd.DataFrame(X_scaled, columns=['환산면적'])
onehot_data_df = pd.DataFrame(onehot.toarray())

X_data_df = pd.concat([numeric_data_df , onehot_data_df], axis=1)
X_data_df

,환산면적,0,1,2,3,4,5,6,7,8,...,171,172,173,174,175,176,177,178,179,180
0,0.132999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.278283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.332455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.332455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.205973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8566,0.688582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8567,0.638286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8568,0.698947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8569,0.698947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
X_data = np.array(X_data_df)
y_data = y_scaled.reshape(-1,)

print(X_data.shape, y_data.shape)

(8571, 182) (8571,)


# K-Fold

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

params ={
    'n_estimators':[100, 200, 300, 500],
    'max_depth':[6,8,15,None],
}

rf = RandomForestRegressor(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf, param_grid=params, cv=5, n_jobs=-1)
grid_cv.fit(X_data, y_data)

In [104]:
print(grid_cv.best_params_)

{'max_depth': 8, 'n_estimators': 500}


## GBM

In [18]:
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

seed = 1234
kf = KFold(n_splits=10, shuffle=True, random_state=seed)

tuple_list = []
for i, (train_idx, test_idx) in enumerate(kf.split(X_data)):
    print('Fold {}'.format(i))
    
    X_train, y_train = X_data[train_idx], y_data[train_idx]
    X_test, y_test = X_data[test_idx], y_data[test_idx]

    # GBM
    for j in [300, 500, 1000]:
        gbm = GradientBoostingRegressor(n_estimators=j)
        gbm.fit(X_train, y_train)
        preds = gbm.predict(X_test)

        mse = mean_squared_error(y_test, preds)
        tuple_list.append((i,j,mse))

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9


In [19]:
MSE_df = pd.DataFrame(columns = ['Fold', 'n_estimators', 'MSE'])
for i, j, mse in tuple_list:
    MSE_df = MSE_df.append({'Fold' : i,
                            'n_estimators' : j,
                            'MSE' : mse}, ignore_index=True)
MSE_df.head(10)

,Fold,n_estimators,MSE
0,0.0,300.0,0.003577
1,0.0,500.0,0.003249
2,0.0,1000.0,0.002843
3,1.0,300.0,0.003596
4,1.0,500.0,0.003279
5,1.0,1000.0,0.003072
6,2.0,300.0,0.003404
7,2.0,500.0,0.003185
8,2.0,1000.0,0.003039
9,3.0,300.0,0.004742


In [20]:
MSE_df.groupby('n_estimators').mean().sort_values(by='MSE')

,Fold,MSE
n_estimators,,
1000.0,4.5,0.003061
500.0,4.5,0.003335
300.0,4.5,0.003623


## XGBoost

In [94]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

xgb_reg = xgb.XGBRegressor()
parameters = {'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb_reg,
                        parameters,
                        cv = 5,
                        verbose=False)

xgb_grid.fit(X_data, y_data)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_para

In [96]:
print(xgb_grid.best_params_)

{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 500, 'subsample': 0.7}


## Ensemble

In [114]:
import torch
from sklearn.metrics import mean_squared_error

class EnsembleRegressor:
    def __init__(self, models, weights=None):
        
        self.models = models
        self.weights = weights
        
    def predict(self, X_test):
        pred_list = []
        for i, model in enumerate(models.values()):
            if i == 0:
                pred_list.append(model(torch.tensor(X_test.astype(np.float32))).detach().numpy().reshape(-1,))
            else:
                pred_list.append(model.predict(X_test))
        preds = np.array(pred_list)
            
        prediction = np.average(preds, axis=0, weights=self.weights)

        return prediction
    
    def random_search(self, X_test, y_test, num_search):
        weights = torch.softmax(torch.rand(num_search, len(self.models)), axis=1).numpy()
        
        pred_list = []
        for i, model in enumerate(models.values()):
            if i == 0:
                pred_list.append(model(torch.tensor(X_test.astype(np.float32))).detach().numpy().reshape(-1,))
            else:
                pred_list.append(model.predict(X_test))
        preds = np.array(pred_list)

        mse = []
        for i in range(num_search):
            prediction = np.average(preds, axis=0, weights=weights[i])
            mse_ = mean_squared_error(y_test, prediction)
            mse.append(mse_)
        
        result = pd.DataFrame(weights, columns=['w1', 'w2', 'w3', 'w4', 'w5', 'w6'])
        result['MSE'] = mse

        return result.sort_values('MSE', ascending=True)

In [30]:
from sklearn.model_selection import KFold

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
from torch.optim import Adam
import tqdm

from model import Model
from train import Trainer

from sklearn.metrics import mean_squared_error

seed = 1234
kf = KFold(n_splits=10, shuffle=True, random_state=seed)

MSE_list = []
for i, (train_idx, test_idx) in enumerate(kf.split(X_data)):
    print('Fold {}'.format(i))
    
    X_train, y_train = X_data[train_idx], y_data[train_idx]
    X_test, y_test = X_data[test_idx], y_data[test_idx]
    
    # Dicision Tree
    dt = DecisionTreeRegressor(random_state=seed)
    dt.fit(X_train, y_train)
    
    # RandomForest
    rf = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=seed)
    rf.fit(X_train, y_train)
    
    # GBM
    gbm = GradientBoostingRegressor()
    gbm.fit(X_train, y_train)
    
    # XGBoost
    xgb_reg = xgb.XGBRegressor(learning_rate=0.1, max_depth=10, n_estimators=500)
    xgb_reg.fit(X_train, y_train, verbose=False)

    # LightGBM
    lgb_reg = lgb.LGBMRegressor(n_estimators=500)
    lgb_reg.fit(X_train, y_train,
                eval_set=[(X_test, y_test)],
                eval_metric='l1',
                early_stopping_rounds=1000,
                verbose = False)
    
    # Neural Network
    dX_train, dy_train = torch.tensor(X_train), torch.tensor(y_train)
    dX_test, dy_test = torch.tensor(X_test), torch.tensor(y_test)

    train_dataset = TensorDataset(dX_train, dy_train)
    test_dataset = TensorDataset(dX_test, dy_test)
    
    BATCH_SIZE = 128
    train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
    test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False)
    
    model = Model(input_size = dX_train.shape[1], hidden_size = 128)
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    
    MSE = nn.MSELoss(reduction = 'mean')
    trainer = Trainer(model, train_loader, test_loader, criterion = MSE, optimizer = Adam,
                  epochs = 60, lr = 0.0007, l2_norm =0.03, device = device)
    trainer.fit()
    
    model = trainer.get_best_model()
    
    # Ensemble
    models = {'DNN' : model, 'DecisionTree' : dt, 'RandomForest' : rf, 'GBM' : gbm, 'XGBOOST': xgb_reg, 'LightGBM' : lgb_reg}

    ensemble = EnsembleRegressor(models, weights=None)
    result = ensemble.random_search(X_test, y_test, 30000)
    optimal_weight = result.iloc[0,:-1].tolist()
    
    ensemble = EnsembleRegressor(models, weights=optimal_weight)
    preds = ensemble.predict(X_test)
    
    mse = mean_squared_error(y_test, preds)
    MSE_list.append(mse)

Fold 0


  0%|          | 0/61 [00:00<?, ?it/s]

epoch: 1/60, train_loss: 0.03188798, val_loss: 0.00932121
epoch: 2/60, train_loss: 0.01367551, val_loss: 0.00671258


  0%|          | 0/61 [00:00<?, ?it/s]

epoch: 3/60, train_loss: 0.00917223, val_loss: 0.00711118
epoch: 4/60, train_loss: 0.00759330, val_loss: 0.00611583


...

epoch: 55/60, train_loss: 0.00275749, val_loss: 0.00403219
epoch: 56/60, train_loss: 0.00244796, val_loss: 0.00419183


  0%|          | 0/61 [00:00<?, ?it/s]

epoch: 57/60, train_loss: 0.00259357, val_loss: 0.00412471
epoch: 58/60, train_loss: 0.00259937, val_loss: 0.00405558


100%|██████████| 61/61 [00:00<00:00, 346.29it/s]


epoch: 59/60, train_loss: 0.00265818, val_loss: 0.00393336
epoch: 60/60, train_loss: 0.00250371, val_loss: 0.00402431
best mse : 0.00378816


In [31]:
result = pd.DataFrame(MSE_list, columns=['MSE'])
print(result)
print()
print(result.mean())
print(result.std())

        MSE
0  0.002392
1  0.002208
2  0.002605
3  0.003102
4  0.003382
5  0.002470
6  0.001908
7  0.001816
8  0.002911
9  0.002339

MSE    0.002513
dtype: float64
MSE    0.000501
dtype: float64
